In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()


True

In [15]:
from langchain_groq import ChatGroq
api_key=os.getenv("GROQ_APIKEY")
llm=ChatGroq(api_key=api_key,model_name="meta-llama/llama-4-scout-17b-16e-instruct")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000002716CF49C50>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000002716FF8E590>, model_name='meta-llama/llama-4-scout-17b-16e-instruct', model_kwargs={}, groq_api_key=SecretStr('**********'))

Stuff Documents

In [16]:
from langchain.chains import LLMChain
from langchain import PromptTemplate
from langchain_community.document_loaders import PyMuPDFLoader
loader=PyMuPDFLoader("apjspeech.pdf")
docs=loader.load_and_split()
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [17]:
template=''' 
Write a concise and a short summary of the following speech: {text}
'''
prompt=PromptTemplate(input_variables=['text'],template=template)


In [18]:
from langchain.chains.summarize import load_summarize_chain


In [19]:
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run(docs)
output_summary  



> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 
Write a concise and a short summary of the following speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, 

"Here is a concise and short summary of A.P.J. Abdul Kalam's departing speech:\n\n**Summary:**\n\nFormer President A.P.J. Abdul Kalam reflects on his five-year tenure, highlighting key experiences and messages that guided his presidency. He emphasizes the importance of:\n\n* Empowering youth, villages, and farmers\n* Promoting education, healthcare, and connectivity\n* Overcoming challenges and calamities through partnership and courage\n* Defending the nation and preserving cultural heritage\n* Fostering a developed India by 2020, with a focus on equitable growth, sustainability, and social justice.\n\n**Key Takeaways:**\n\n* The youth are the future wealth of India, and their aspirations should guide the nation's development.\n* Empowering villages and rural areas is crucial for bridging the rural-urban divide.\n* Education, healthcare, and connectivity are essential for societal transformation.\n* India can become a developed nation by 2020 with collective effort, determination, and

MAP REDUCE


In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter


In [21]:
docs

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [22]:
final_documents=RecursiveCharacterTextSplitter(chunk_size=2000,chunk_overlap=100).split_documents(docs)
final_documents

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'source': 'apjspeech.pdf', 'file_path': 'apjspeech.pdf', 'total_pages': 7, 'format': 'PDF 1.3', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'subject': '', 'keywords': '', 'moddate': 'D:20070730160943', 'trapped': '', 'modDate': 'D:20070730160943', 'creationDate': 'D:20070730160943', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I \nenjoyed every minute of my tenure enriched by the wonderful association from each one \nof you, hailing from different walks of life, be it politics, science and technology, \nacademics, arts, literature, business, judiciary, administration,

In [23]:
len(final_documents)

13

In [24]:
chunk_prompt='''
Please summarize the below speech:
Speech:'{text}'
Summary:
'''
map_prompt=PromptTemplate(input_variable=['text'],template=chunk_prompt)

In [25]:
final_prompt='''
Provide a final summary of the entire speech with these important points.
Add a motivation title and start the precise summary with an introduction.
Speech:{text}
'''
final_prompt_temp=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_temp

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide a final summary of the entire speech with these important points.\nAdd a motivation title and start the precise summary with an introduction.\nSpeech:{text}\n')

In [26]:
summary_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt,
    combine_prompt=final_prompt_temp,
    verbose=True
)
output=summary_chain.run(final_documents)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:'A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narrate, how I 
enjoyed every minute of my tenure enriched by the wonderful association from each one 
of you, hailing from different walks of life, be it politics, science and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and student 
community who are the future wealth of our country. During my interaction at 
Rashtrapati Bhavan in Delhi and at every state and union territory as well as through my 
online interactions, I have many uni


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:

Provide a final summary of the entire speech with these important points.
Add a motivation title and start the precise summary with an introduction.
Speech:Here is a summary of the speech:

Former President of India, A.P.J. Abdul Kalam, is bidding farewell after completing his five-year term. He expresses his gratitude and delight in having worked with people from diverse backgrounds. He shares his experiences and key messages that he conveyed during his tenure, which include:

1. Accelerating development to meet the aspirations of youth
2. Empowering villages
3. Mobilizing rural resources for competitiveness
4. Focusing on agriculture growth from seed to food
5. Overcoming problems through partnership and courage
6. Building connectivity for societal transformation
7. Defending the nation with pride
8. Fostering a youth movement for a developed India by 2020

He recalls a significant question from a school

'**Empowering a Developed India: A Vision for the Future**\n\n**Introduction**\n\nThe speech delivered by Former President of India, A.P.J. Abdul Kalam, is a reflective and inspirational message highlighting his priorities and vision for India\'s development. As he bids farewell after completing his five-year term, he expresses his gratitude and delight in having worked with people from diverse backgrounds.\n\n**Summary of Key Points**\n\nThe speech emphasizes the importance of:\n\n1. **Accelerating development** to meet the aspirations of youth\n2. **Empowering villages** and mobilizing rural resources for competitiveness\n3. **Focusing on agriculture growth** from seed to food\n4. **Overcoming problems** through partnership and courage\n5. **Building connectivity** for societal transformation\n6. **Defending the nation** with pride\n7. **Fostering a youth movement** for a developed India by 2020\n\nThe speaker shares inspiring stories of resilience and courage, including:\n\n* A scho